# **LIGHT** 



In [ ]:
import tensorflow as tf #Импортируем tensorflow
import datetime, os #Для подсчета времени и работы с файловой системой
import numpy as np #Для работы с матрицами 
from tensorflow.keras import utils #Для работы с категориальными данными 
import sys #Для специльного вывода
import matplotlib.pyplot as plt #Для виузализации 
import random #Для генерации случайных чисел 

**Универсальная функция для формирования датасета** https://www.youtube.com/watch?v=ljMOTWrszxQ&t=326s


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
X_train.shape

In [ ]:
y_train[1]

In [ ]:
#Функция формирования датсета mnist 
#flatten - вытягивание картинки из формы 28, 28 в 784
def data_cifar10(flatten = True):
  #Скачивааем датасет 
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    
  if(flatten):#Для полносвязной сети  
    X_train = X_train.reshape(-1, 32*32*3)
    X_test = X_test.reshape(-1, 32*32*3)
  else: #Для сверточной сети 
    X_train = X_train.reshape(-1, 32, 32, 3)
    X_test = X_test.reshape(-1, 32, 32, 3)
  #Первеодим во float
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  #Переводим все значения в диапазон от 0 до 1
  X_train /= 255
  X_test /= 255

  print('Форма x_train:', X_train.shape)
  print(X_train.shape[0], 'обучающих примеров')
  print(X_test.shape[0], 'проверочных примеров')
  #Переводим в OneHot
  y_train = utils.to_categorical(y_train, 10).astype(np.float32)
  y_test = utils.to_categorical(y_test, 10).astype(np.float32)
  #Возвращаем датасет
  return X_train, y_train, X_test, y_test #Возвращаем датасет 

**Загрузка данных** https://www.youtube.com/watch?v=ljMOTWrszxQ&t=427s


In [ ]:
x_train, y_train, x_test, y_test = data_cifar10() #Загружаем датасет 

**Параметры обучения и оптимизации** https://www.youtube.com/watch?v=ljMOTWrszxQ&t=444s


In [ ]:
classes = {
    0 : 'airplane',
    1 : 'automobile',
    2 : 'bird',
    3 : 'cat',
    4 : 'deer',
    5 : 'dog',
    6 : 'frog',
    7 : 'horse',
    8 : 'ship',
    9 : 'truck'
}

## **Сверточные нейронные сети** 


In [ ]:
padding = "SAME"  
num_output_classes = 10
batchSize = 256
epochs = 100
learningRate = 0.001 

In [ ]:
leaky_relu_alpha = 0.2
dropout_rate = 0.5 

def conv2d( inputs , filters , stride_size ): #Слой для создания сверточного слоя 
    out = tf.nn.conv2d( inputs , filters , strides=[ 1 , stride_size , stride_size , 1 ] , padding=padding ) 
    return tf.nn.leaky_relu(out , alpha=leaky_relu_alpha ) 

def maxpool( inputs , pool_size , stride_size ): #Слой для применения maxpooling
    return tf.nn.max_pool2d(inputs , ksize=[ 1 , pool_size , pool_size , 1 ] , padding='VALID' , strides=[ 1 , stride_size , stride_size , 1 ] )

def dense(inputs , weights): #Слой для создания полносвязного слоя 
    x = tf.nn.leaky_relu(inputs @ weights, alpha=leaky_relu_alpha )
    return tf.nn.dropout( x , rate=dropout_rate )

In [ ]:
output_classes = 10 #Определяем число классов  https://www.youtube.com/watch?v=ljMOTWrszxQ&t=3216s
initializer = tf.initializers.glorot_uniform() #Инициализатор переменных по форме
def get_weight(  shape, name ): #Функция для получения весов
    return tf.Variable(initializer(shape) , name=name , trainable=True , dtype=tf.float32 )

#Формы слоев 
shapes = [
    [ 1 , 1 , 3 , 16 ] , 
    [ 2 , 2 , 16 , 16 ] , 
    [ 2 , 2 , 16, 32 ] , 
    [ 2 , 2 , 32 , 32 ] ,
    [ 2 , 2 , 32 , 64 ] , 
    [ 2 , 2 , 64 , 64 ] ,
    [ 1024 , 32 ] , 
    [ 32 , output_classes ] ,
]

#Создание весов https://www.youtube.com/watch?v=ljMOTWrszxQ&t=3799s
weights = []
for i in range(len(shapes)):
    weights.append( get_weight(shapes[i] , 'weight{}'.format( i )))


In [ ]:
#Модель https://www.youtube.com/watch?v=ljMOTWrszxQ&t=3846s 
def model( x ) :
    x = tf.cast( x , dtype=tf.float32 )
    c1 = conv2d( x , weights[0] , stride_size=1 ) 
    c1 = conv2d( c1 , weights[1] , stride_size=1 )
    p1 = maxpool( c1 , pool_size=2 , stride_size=2 )
    
    c2 = conv2d( p1 , weights[2] , stride_size=1 )
    c2 = conv2d( c2 , weights[3] , stride_size=1 ) 
    p2 = maxpool( c2 , pool_size=2 , stride_size=2 )
    
    c3 = conv2d( p2 , weights[4] , stride_size=1 ) 
    c3 = conv2d( c3 , weights[5] , stride_size=1 ) 
    p3 = maxpool( c3 , pool_size=2 , stride_size=2)
    flatten = tf.reshape( p3 , shape=( tf.shape( p3 )[0] , -1 ))

    d1 = dense( flatten , weights[6])
    logits = tf.matmul( d1 , weights[7] )

    return tf.nn.softmax(logits)


In [ ]:
def loss( pred , target ): #Функция подсчета ошибки  https://www.youtube.com/watch?v=ljMOTWrszxQ&t=4107s
    return tf.losses.categorical_crossentropy( target , pred )

optimizer = tf.optimizers.Adam(learningRate)

def train( model, inputs , outputs ):
    m = tf.keras.metrics.Accuracy() #Задаем метрику
    with tf.GradientTape() as tape:
        current_loss = loss( model( inputs ), outputs)
    # Градиентный спуск. Инициализируем через learning rate
    # Функция реализует градиентный спуск и обратное распространение ошибки.
    grads = tape.gradient( current_loss , weights )
    #Применение градиентного спуска
    optimizer.apply_gradients( zip( grads , weights ) )
    #Подсчет точности сети
    m.update_state(np.argmax(outputs, axis=1), np.argmax(model(inputs), axis=1))
    return tf.reduce_mean(current_loss) , m.result().numpy()

In [ ]:
path = '/tmp/mylogs/eager/' #Указываем путь для сохранения даных для Tensorboard https://www.youtube.com/watch?v=ljMOTWrszxQ&t=4166s

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #Получаем информацию о текущем времени для добавления данной информации к имени  
loss_log_dir = path + current_time + '/data' #Создаем папку с именем из текущего времени
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir) #Создаем средство записи файла резюме для данного каталога

amount_bathces = int(len(x_train) / batchSize) #Считаем число батчей для каждой эпохи. Понадобится для вывода

with loss_summary_writer.as_default(): #Используя созданное средство для записи файла в резюме 
  
  for epoch in range(1, epochs + 1):
    learningEpochStartTime = datetime.datetime.now() #Запоминаем время начала эпохи 
    print('Эпоха', epoch, '/', epochs) #Пишем текущую эпоху и общее число эпох
    avg_loss = 0 #Задаем среднюю ошибку 
    for batch in range(0, len(x_train), batchSize):
      current_loss, accuracy = train( model , x_train[batch:batch + batchSize] , y_train[batch:batch + batchSize] )
      avg_loss += current_loss

      params = {'Длительность обучения на эпохе: ': datetime.datetime.now() - learningEpochStartTime, #Считаем время обучения на данной эпохе и добавляем в словарь
                'loss: ': current_loss.numpy(), #Переводим ошибку в Numpy и добавляем в словарь
                'accuracy: ': accuracy} #Добавляем точность в словарь
      if(batch >= len(x_train) - batchSize): #На последнем батче
        params['loss: '] = (avg_loss / amount_bathces).numpy() #Выводим среднюю ошибку на всей эпохе
      current_batch = int(batch / batchSize) + 1 #Считаем номер текущего батча
      print_log(current_batch, amount_bathces, params)
    tf.summary.scalar("avg_loss", avg_loss, step=epoch) #Сохраняем данные для Tensorboard
    tf.summary.scalar("accuracy", accuracy, step=epoch) #Сохраняем данные для Tensorboard
    loss_summary_writer.flush() #Очищаем буфер вывода 
    print() #Вручную переносим каретку на следующую строку, чтобы не стирать финальные значения сети на эпохе


In [ ]:
#Выводим для примера картинки по каждому классу https://www.youtube.com/watch?v=ljMOTWrszxQ&t=4402s
print('Сеть распознала цифры: ')
fig, axs = plt.subplots(1, 10, figsize=(25, 3)) #Создаем полотно из 10 графиков
for i in range(10): #Проходим по классам от 0 до 9
  label_indexes = np.where(np.argmax(y_test, axis = 1)==i)[0] #Получаем список из индексов положений класса i в y_test
  index = random.choice(label_indexes) #Случайным образом выбираем из списка индекс
  img = x_test[index] #Выбираем из x_train нужное изображение
  axs[i].imshow(img.reshape(32, 32, 3)) #Отображаем изображение i-ым графиков
  print(classes[np.argmax(model([img]))], end=' ')
plt.show() #Показываем изображения

## Визуализация


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir '/tmp/mylogs/eager/'

**Тренируемые параметры** https://www.youtube.com/watch?v=ljMOTWrszxQ&t=957s


In [ ]:
trainableParams = [] #Лист тренируемых параметров
# Объявляем веса, W1
trainableParams.append(tf.Variable(tf.random.normal([3072, 300], stddev=0.03), name='W1')) #300 размер скрытого слоя, инциалзириуем значения

# Используя нормальное распределение со средним ноль и статистическим отклонением 0.03, определяем bias(Аналогичная есть у numpy)
trainableParams.append(tf.Variable(tf.random.normal([300], stddev=0.03), name='b1'))

# То же делаем для весов и bias(отклонения) от скрытого к выходному
trainableParams.append(tf.Variable(tf.random.normal([300, 10], stddev=0.03), name='W2'))
trainableParams.append(tf.Variable(tf.random.normal([10]), name='b2'))

In [ ]:
#Функция подсчета ошибки https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1151s
def loss(pred , target):
    return tf.losses.categorical_crossentropy(target , pred)

In [ ]:
#Полносвязный слой с несколькими функциями активации https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1291s
def base(x , params):
    W1 = params[0]
    b1 = params[1]
    W2 = params[2]
    b2 = params[3]

    #a @ b - аналог tf.matmul(a, b)
    hiddenOut = tf.nn.relu(x@W1+b1) #Умножаем вход на веса W1, прибавляем b1 и применяем функцию активации relu
    y = tf.nn.softmax(hiddenOut@W2+b2) #Умножаем выход скрытого слоя на веса W2, прибавляем b2 и применяем функцию активации softmax
    return y

In [ ]:
#Модель  https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1541s
def model(x):
    y = base(x, trainableParams)
    return y 


In [ ]:
x_train[1].shape

In [ ]:
#Посмотрим, какой выход у модели, если на вход подать случайную картинку 
model(x_train[[1]]) #Смотрим вывод сети для первого изображения из MNIST https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1559s

In [ ]:
y_train[1]

In [ ]:
#Функция для информативного вывода обучения https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1665s
#На вход получает:
# current - номер текущего батча
# amount - число всех батчей 
# params - словарь дополнительных параметров для вывода 

def print_log(current, amount, params):
  #Мы будем выводить прогрессбар, подобный прогрессбару при обучении нейронных сетей на Keras
  #Формат прогрессбара 23/120 [=====>------------------------]
  
  bar_len = 30 #Длина бара 
  percent = int(current * bar_len / amount) #Процент выполненной работы
  progressbar = ''

  for i in range(bar_len): #Проходим по всем элементам прогрессбара и добавляем символы в соответсвии с прогресом 
    if(i < percent):
      progressbar += '='
    elif(i == percent):
      progressbar += '>'
    else:
      progressbar += '-'

  #Добавялем в финальное сообщение символ переноса коретки консоли на начальную строку, добавляпем информацию о номере батча
  #количестве всех батчей, прогрессбар
  #Символ переноса коретки \r добавляется для того, чтобы каждый новый батч перезаписывать вывод. Таким образом вывод не будет засоряться повторяющейся информацией
  message = "\r" + str(current) + '/' + str(amount) + ' [' + progressbar + ']  ' 
  #Добавляем дополнительные параметры в вывод
  for key in params:
    message += key + str(params[key]) + '. '
  
  print(message, end='')

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate) #Задаем оптимизатор https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1860s
m = tf.keras.metrics.Accuracy() #Задаем метрику

def train(model, inputs, outputs): #Функция тренировки сети 

    with tf.GradientTape() as tape:
      current_loss = tf.reduce_mean(loss(model(inputs), outputs)) #Считаем ошибку

      # Градиентный спуск. Инициализируем через learning rate
      # Функция реализует градиентный спуск и обратное распространение ошибки.
      grads = tape.gradient(current_loss , trainableParams)
      #Применение градиентного спуска
      optimizer.apply_gradients(zip(grads , trainableParams))
      #Подсчет точности сети 
      m.update_state(np.argmax(outputs, axis=1), np.argmax(model(inputs), axis=1))
    return current_loss, m.result().numpy()

In [ ]:
!rm -R /tmp/mylogs/eager #Очищаем папку с записанной информацией обучения (если код запускается не в первый раз)

In [ ]:
path = '/tmp/mylogs/eager/' #Указываем путь для сохранения даных для Tensorboard https://www.youtube.com/watch?v=ljMOTWrszxQ&t=2237s


current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #Получаем информацию о текущем времени для добавления данной информации к имени  
loss_log_dir = path + current_time + '/data' #Создаем папку с именем из текущего времени
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir) #Создаем средство записи файла резюме для данного каталога

amount_bathces = int(len(x_train) / batchSize) #Считаем число батчей для каждой эпохи. Понадобится для вывода

with loss_summary_writer.as_default(): #Используя созданное средство для записи файла в резюме 
 
  for epoch in range(1, epochs + 1): #Проходим по каждой эпохе
    learningEpochStartTime = datetime.datetime.now() #Запоминаем время начала эпохи 
    print('Эпоха', epoch , '/', epochs) #Пишем текущую эпоху и общее число эпох
    avg_loss = 0 #Задаем среднюю ошибку

    for batch in range(0, len(x_train), batchSize): #Проходим по x_train с шагом batchSize
      current_loss, accuracy = train(model, x_train[batch: batch + batchSize], y_train[batch: batch + batchSize]) #Тренируем сеть и получаем значение ошибки
      avg_loss += current_loss #Считаем среднюю ошибку

      #Задаем параметры, которые будем выводить 
      params = {'Время обучения на эпохе: ': datetime.datetime.now() - learningEpochStartTime, #Считаем время обучения на данной эпохе и добавляем в словарь
                'loss: ': round(current_loss.numpy(), 4), #Переводим ошибку в Numpy и добавляем в словарь
                'accuracy: ': round(accuracy, 4)} #Добавляем точность в словарь
      if(batch >= len(x_train) - batchSize): #На последнем батче
        params['loss: '] = round((avg_loss / amount_bathces).numpy(), 4) #Выводим среднюю ошибку на всей эпохе

      current_batch = int(batch / batchSize) + 1 #Считаем номер текущего батча
      print_log(current_batch, amount_bathces, params) #Выводим всю нужную информацию 
    
    tf.summary.scalar("avg_loss", avg_loss, step=epoch) #Сохраняем данные для Tensorboard
    tf.summary.scalar("accuracy", accuracy, step=epoch) #Сохраняем данные для Tensorboard
    loss_summary_writer.flush() #Очищаем буфер вывода 
    print() #Вручную переносим каретку на следующую строку, чтобы не стирать финальные значения сети на эпохе

In [ ]:
#Выводим для примера картинки по каждому классу https://www.youtube.com/watch?v=ljMOTWrszxQ&t=2481s
print('Сеть распознала цифры: ')
fig, axs = plt.subplots(1, 10, figsize=(25, 3)) #Создаем полотно из 10 графиков
for i in range(10): #Проходим по классам от 0 до 9
  label_indexes = np.where(np.argmax(y_test, axis = 1)==i)[0] #Получаем список из индексов положений класса i в y_test
  index = random.choice(label_indexes) #Случайным образом выбираем из списка индекс
  img = x_test[index] #Выбираем из x_train нужное изображение
  axs[i].imshow(img.reshape(32, 32, 3)) #Отображаем изображение i-ым графиков
  print(np.argmax(model([img])), end=' ')
plt.show() #Показываем изображения

#PRO Оценка стоимости квартир


Парсинг данных и архитектуру сети взял с занятия по регрессии

In [ ]:
import pandas as pd
from google.colab import files
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output

from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/19 Регрессия/moscow (1).csv', sep=";") 
df = df.iloc[::2,:] #Выбираем нечётные строки, в чётных строках в исходном фрейме пустые строки для комментариев
data = df.values #Вытаскиваем данные в numpy array

In [ ]:
df.head(20) #Показываем пример данных (первые 6 колонок)

In [ ]:
print(df.columns) #Показываем имена колонок данных
#Мы будем использовать

#В xTrain - база числовых значени
#Комнат
#Метро / ЖД станции
#От станции
#Дом
#Балкон
#Санузел
#Площадь

#В xTrainC - база текстов о квартирах
#Примечание

#В yTrain
#Цена, руб.

#Остальные колонки игнорируем

##Функции парсинга

*Разбор данного раздела:* https://youtu.be/PCMkFDuxvCI?t=5834


In [ ]:
#Во всех функция дальше
#d - описание квартиры, одна строка из data1

#Вычисляем количество комнат
#maxRoomCount - максимальное число комнат в квартире
def getRoomsCount(d, maxRoomCount):
  roomsCountStr = d[0] #Получаем строку с числом комнат

  roomsCount = 0
  try:
    roomsCount = int(roomsCountStr) #Пробуем превратить строку в число
    if (roomsCount > maxRoomCount): 
      roomsCount = maxRoomCount #Если число комнат больше максимального, то присваиваем максимальное
  except: #Если не получается превратить строку в число
    if (roomsCountStr == roomsCountStr): #Проверяем строку на nan (сравнение с самим собой)
      if ("Ст" in roomsCountStr): #Еcть строка = "Ст", значит это Студия
        roomsCount = maxRoomCount + 1

  return roomsCount

#Превращаем число комнат в категорию
def getRoomsCountCategory(d, maxRoomCount):
  roomsCount = getRoomsCount(d, maxRoomCount) #Получаем число комнат
  roomsCount = utils.to_categorical(roomsCount, maxRoomCount+2) #Превращаем в категорию
  #maxRoomCount+2 потому что 0 зарезервирован на неопознаное число комнат, а maxRoomCount+1 на "Студию"
  return roomsCount

#Получаем индекс станции метро
#allMetroNames - все уникальные названия метро в базе
def getMetro(d, allMetroNames):
  metroStr = d[1] #Получаем строку метро
  metro = 0
  
  if (metroStr in allMetroNames): #Если находим метро во всех названиях
    metro = allMetroNames.index(metroStr)+1 #Присваиваем индекс
    #+1 так как 0 зарезервирован на неопознанное метро
    
  return metro

#Получаем тип метро
#0 - внутри кольца
#1 - кольцо
#2 - 1-3 станции от конца
#3 - 4-8 станций от кольца
#4 - больше 8 станций от кольца
def getMetroType(d):
  metroTypeStr = d[1] #Получаем строку метро
  metroTypeClasses = 5 #Число классов метро
  metroType = metroTypeClasses - 1 #Изначально считаем последний класс
  
  #Метро внутри кольца
  metroNamesInsideCircle = ["Площадь Революции", "Арбатская", "Смоленская", "Красные Ворота", "Чистые пруды", "Лубянка", "Охотный Ряд", "Библиотека имени Ленина", "Кропоткинская", "Сухаревская", "Тургеневская", "Китай-город", "Третьяковская", "Трубная", "Сретенский бульвар", "Цветной бульвар", "Чеховская", "Боровицкая", "Полянка", "Маяковская", "Тверская", "Театральная", "Новокузнецкая", "Пушкинская", "Кузнецкий Мост", "Китай-город", "Александровский сад"]
  #Метро на кольце
  metroNamesCircle = ["Киевская", "Парк Культуры", "Октябрьская", "Добрынинская", "Павелецкая", "Таганская", "Курская", "Комсомольская", "Проспект Мира", "Новослободская", "Белорусская", "Краснопресненская"]
  #Метро 1-3 станции от кольца
  metroNames13FromCircle = ["Бауманская", "Электрозаводская", "Семёновская", "Площадь Ильича", "Авиамоторная", "Шоссе Энтузиастов", "Римская", "Крестьянская Застава", "Дубровка", "Пролетарская", "Волгоградский проспект", "Текстильщики", "Автозаводская", "Технопарк", "Коломенская", "Тульская", "Нагатинская", "Нагорная", "Шаболовская", "Ленинский проспект", "Академическая", "Фрунзенская", "Спортивная", "Воробьёвы горы", "Студенческая", "Кутузовская", "Фили", "Парк Победы", "Выставочная", "Международная", "Улица 1905 года", "Беговая", "Полежаевская", "Динамо", "Аэропорт", "Сокол", "Деловой центр", "Шелепиха", "Хорошёвская", "ЦСКА", "Петровский парк", "Савёловская", "Дмитровская", "Тимирязевская", "Достоевская", "Марьина Роща", "Бутырская", "Фонвизинская", "Рижская", "Алексеевская", "ВДНХ", "Красносельская", "Сокольники", "Преображенская площадь"]
  #Метро 4-8 станций от кольа
  metroNames48FromCircle = ["Партизанская", "Измайловская", "Первомайская", "Щёлковская", "Новокосино", "Новогиреево", "Перово", "Кузьминки", "Рязанский проспект", "Выхино", "Лермонтовский проспект", "Жулебино", "Партизанская", "Измайловская", "Первомайская", "Щёлковская", "Новокосино", "Новогиреево", "Перово", "Кузьминки", "Рязанский проспект", "Выхино", "Лермонтовский проспект", "Жулебино", "Улица Дмитриевского", "Кожуховская", "Печатники", "Волжская", "Люблино", "Братиславская", "Коломенская", "Каширская", "Кантемировская", "Царицыно", "Орехово", "Севастопольская", "Чертановская", "Южная", "Пражская", "Варшавская", "Профсоюзная", "Новые Черёмушки", "Калужская", "Беляево", "Коньково", "Университет", "Багратионовская", "Филёвский парк", "Пионерская", "Кунцевская", "Молодёжная", "Октябрьское Поле", "Щукинская", "Спартак", "Тушинская", "Сходненская", "Войковская", "Водный стадион", "Речной вокзал", "Беломорская", "Ховрино", "Петровско-Разумовская", "Владыкино", "Отрадное", "Бибирево", "Алтуфьево", "Фонвизинская", "Окружная", "Верхние Лихоборы", "Селигерская", "ВДНХ", "Ботанический сад", "Свиблово", "Бабушкинская", "Медведково", "Преображенская площадь", "Черкизовская", "Бульвар Рокоссовского"]
  
  #Проверяем, в какую категорию попадает наша станция
  if (metroTypeStr in metroNamesInsideCircle):
    metroType = 0
  if (metroTypeStr in metroNamesCircle):
    metroType = 1
  if (metroTypeStr in metroNames13FromCircle):
    metroType = 2
  if (metroTypeStr in metroNames48FromCircle):
    metroType = 3
  
  #Превращаем результат в категорию
  metroType = utils.to_categorical(metroType, metroTypeClasses)
  return metroType


#Вычисляем растояние до метро
def getMetroDistance(d):
  metroDistanceStr = d[2] #Получаем строку
  
  metroDistance = 0 #Расстояние до метро
  metroDistanceType = 0 #Тип расстояния - пешком или на транспорте
  
  #ЕСли строка не равна nan  
  if (metroDistanceStr == metroDistanceStr):
    if (len(metroDistanceStr) > 0):
      #Определяем тип расстояния
      if (metroDistanceStr[-1] == "п"):
        metroDistanceType = 1 #Пешком
      elif (metroDistanceStr[-1] == "т"):
        metroDistanceType = 2 #На транспорте

      #Выбрасываем последний символ, чтобы осталось только число
      metroDistanceStr = metroDistanceStr[:-1]
      try:
        #Разделяем дистанции на категории
        metroDistance = int(metroDistanceStr)
        if (metroDistance < 3):
          metroDistance = 1
        elif (metroDistance < 6):
          metroDistance = 2
        elif (metroDistance < 10):
          metroDistance = 3
        elif (metroDistance < 15):
          metroDistance = 4
        elif (metroDistance < 20):
          metroDistance = 5
        else:
          metroDistance = 6
      except: #Если в строке не число, то категория 0
        metroDistance = 0
  
  #Число классов дистанции
  metroDistanceClasses = 7
  
  #У нас 7 категорий дистанции по расстоянию
  #И 3 типа дистанции - неопознанный, пешком и транспортом
  #Мы создадим вектор длины 3*7 = 21
  #Будем преобразовывать индекс расстояния 0-6 в 0-20
  #Для типа "Пешком" - ничего не меняем
  if (metroDistanceType == 2):
    metroDistance += metroDistanceClasses #Для типа "Транспортом" добавляем 7
  if (metroDistanceType == 0):
    metroDistance += 2*metroDistanceClasses #Для неопознанного типа добавляем 14
    
  #Превращаем в категории
  metroDistance = utils.to_categorical(metroDistance, 3*metroDistanceClasses)
  return metroDistance

#Получаем 4 данных
#- этаж квартиры
#- этажность дома
#- индикатор, что последний этаж
#- тип дома
def getHouseTypeAndFloor(d):
  try:
    houseStr = d[3] #Получаем строку типа дома и этажей
  except:
    houseStr = ""
  
  houseType = 0 #Тип дома
  floor = 0 #Этаж квартиры
  floors = 0 #Этажность дома
  isLastFloor = 0 #Индикатор последнего этажа
  
  #Проверяем строку на nan
  if (houseStr == houseStr):
    if (len(houseStr) > 1):
    
      try:
        slashIndex = houseStr.index("/") #Ищем разделитель /
      except:
        print(houseStr)

      try:
        spaceIndex = houseStr.index(" ") #Ищем разделитель " "
      except:
        print(houseStr)

      #Вытаскиваем строки
      floorStr = houseStr[:slashIndex] #Строка этажа
      floorsStr = houseStr[slashIndex+1:spaceIndex] #Строка этажнгости дома
      houseTypeStr = houseStr[spaceIndex+1:] #Строка типа дома

      #Выбираем категорию этажа
      try:
        floor = int(floorStr) #Превращаем строку в число
        floorSave = floor
        if (floorSave < 5):
          floor = 2
        if (floorSave < 10):
          floor = 3
        if (floorSave < 20):
          floor = 4
        if (floorSave >= 20):
          floor = 5
        if (floorSave == 1): #Первый этаж выделяем в отдельную категорию
          floor = 1 

        if (floor == floors): #Если этаж последний, включаем индикатор последнего этажа
          isLastFloor = 1 
      except:
        floor = 0 #Если строка не парсится в число, то категория этажа = 0 (отдельная)

      #Выбираем категорию этажности дома
      try:
        floors = int(floorsStr) #Превращаем строку в число
        floorsSave = floors
        if (floorsSave < 5):
          floors = 1
        if (floorsSave < 10):
          floors = 2
        if (floorsSave < 20):
          floors = 3
        if (floorsSave >= 20):
          floors = 4
      except:
        floors = 0 #Если строка не парсится в число, то категория этажности = 0 (отдельная)

      #Определяем категорию типа дома
      if (len(houseTypeStr) > 0):
        if ("М" in houseTypeStr): 
          houseType = 1
        if ("К" in houseTypeStr): 
          houseType = 2
        if ("П" in houseTypeStr): 
          houseType = 3
        if ("Б" in houseTypeStr): 
          houseType = 4
        if ("?" in houseTypeStr): 
          houseType = 5
        if ("-" in houseTypeStr): 
          houseType = 6
    
    #Превращаем все категории в one hot encoding
    floor = utils.to_categorical(floor, 6)
    floors = utils.to_categorical(floors, 5)
    houseType = utils.to_categorical(houseType, 7)
    
    
  return floor, floors, isLastFloor, houseType


#Вычисляем тип балкона
def getBalcony(d):
  balconyStr = d[4] #Полуаем строку
  #Выписываем все варианты балконов в базе
  balconyVariants = ['Л', 'Б', '2Б', '-', '2Б2Л', 'БЛ', '3Б', '2Л', 'Эрк', 'Б2Л', 'ЭркЛ', '3Л', '4Л', '*Л', '*Б']
  #Проверяем на nan
  if (balconyStr == balconyStr):
    balcony = balconyVariants.index(balconyStr)+1 #Находим индекс строки балкона во всех строках
  else:
    balcony = 0 #Индекс 0 выделяем на строку nan
  
  #Превращаем в one hot encoding
  balcony = utils.to_categorical(balcony, 16)
  
  return balcony

#Определяем тип санузла
def getWC(d):
  wcStr = d[5] #Получаем строку
  #Выписываем все варианты санузлов в базе
  wcVariants = ['2', 'Р', 'С', '-', '2С', '+', '4Р', '2Р', '3С', '4С', '4', '3', '3Р']
  #Проверяем на nan
  if (wcStr == wcStr):
    wc = wcVariants.index(wcStr)+1 #Находим индекс строки санузла во всех строках
  else:
    wc = 0 #Индекс 0 выделяем на строку nan
  
  #Превращаем в one hot encoding
  wc = utils.to_categorical(wc, 14)
  
  return wc

#Определяем площадь
def getArea(d):
  areaStr = d[6] #Поулачем строку площади
  
  if ("/" in areaStr):
    slashIndex = areaStr.index("/") #Находим разделитель /
    try:
      area = float(areaStr[:slashIndex]) #Берём число до разделителя и превращаем в число
    except:
      area = 0 #Если не получается, возвращаем 0
  else:
    area = 0 #Или если нет разделителя, возвращаем 0
    
  return area

#Полуаем цену
def getCost(d):
  costStr = d[7] #Загружаем строку
  
  try:
    cost = float(costStr) #Пробуем превратить в число
  except:
    cost = 0 #Если не получается, возвращаем 0
  
  return cost

#Получаем комментарий
def getComment(d):
  commentStr = d[-1] #Возвращаем данные из последней колонки
  
  return commentStr

#Объединяем все числовые параметры вместе
def getAllParameters(d, allMetroNames):
  #Загружаем все данные по отдельности
  roomsCountType = getRoomsCountCategory(d, 30)
  metro = getMetro(d, allMetroNames)
  metroType = getMetroType(d)
  metroDistance = getMetroDistance(d)
  floor, floors, isLastFloor, houseType = getHouseTypeAndFloor(d)
  balcony = getBalcony(d)
  wc = getWC(d)
  area = getArea(d)

  #Объединяем в один лист
  out = list(roomsCountType)
  out.append(metro)
  out.extend(metroType)
  out.extend(metroDistance)
  out.extend(floor)
  out.extend(floors)
  out.append(isLastFloor)
  out.extend(houseType)
  out.extend(balcony)
  out.extend(wc)
  out.append(area)
  
  return out

#Генерируем обучающаюу выборку - xTrain
def getXTrain(data):
  
  #Получаем строку во всеми вариантами метро
  allMertroNames = list(df["Метро / ЖД станции"].unique())
  
  #Всевращаем все строки в data1 в векторы параметров и записываем в xTrain
  xTrain = [getAllParameters(d, allMertroNames) for d in data]
  xTrain = np.array(xTrain)
  
  return xTrain

#Генерируем обучающую выборку - yTrain
def getYTrain(data):
  
  #Зашружаем лист всех цен квартир по всем строкам data1
  costList = [getCost(d) for d in data] 
  yTrain = np.array(costList)
  
  return yTrain

##Формируем обучающую выборку

*Разбор данного раздела:* https://youtu.be/PCMkFDuxvCI?t=6452



In [ ]:
oneRoomMask = [getRoomsCount(d, 30) == 1 for d in data] #Делаем маску однокомнатных квартир, принцип (getRoomsCount(d, 30) == 1)
data1 = data[oneRoomMask] #В data1 оставляем только однокомнатные квартиры
print(data.shape)
print(data1.shape)

In [ ]:
xTrain = getXTrain(data1)
yTrain = getYTrain(data1)

In [ ]:
print(xTrain.shape)

##Парсинг текста

*Разбор данного раздела:* https://youtu.be/PCMkFDuxvCI?t=6502


In [ ]:
###########################
# Очистка текста и превращение в набор слов
##########################
def text2Words(text):

  #Удаляем лишние символы
  text = text.replace(".", "")# удаляем лишние символы
  text = text.replace("—", "")
  text = text.replace(",", "")
  text = text.replace("!", "")
  text = text.replace("?", "")
  text = text.replace("…", "")
  text = text.lower() #Переводим в нижний регистр
  
  
  words = [] #Тут будут все слов
  currWord = "" #Тут будет накапливаться текущее слово, между двумя пробелами
  
  #идём по всем символам
  for symbol in text:
    
    if (symbol != "\ufeff"): #Игнорируем системынй символ в начале строки
      if (symbol != " "): #Если символ не пробел
        currWord += symbol #То добавляем вимвол в текущее слово
      else: #Если символ пробел
        if (currWord != ""): 
          words.append(currWord) #Добавляем тккущее слово в список слов
          currWord = "" #И обнуляем текущее слово

  #Добавляем финальное слово, если оно не пустое
  #Если не сделать, то потеряем финальное слово, потому что текст чаще всего заканчивается на не пробел
  if (currWord != ""):
        words.append(currWord)
  
  return words


###########################
# Создание словаря - все слова, упорядоченные по частоте появления
##########################
def createVocabulary(allWords):
  
  #Создаём словарь, в котором будут слова и количество их поялвений во всём текста
  #Ключи - все наши слова
  #Количество появлений пока везде 0
  wCount = dict.fromkeys(allWords, 0)

  #Проходим по всем словам
  for word in allWords:
    wCount[word] += 1 #И увеличиаем количество появлений текущего слова на 1

  #Выцепляем лист из словаря
  wordsList = list(wCount.items())
  #И сортируем по частоте появления
  wordsList.sort(key = lambda i:i[1], reverse=1)
  #key = lambda i:i[1] - говорит, что сортировать надо по частоте появления
  #В i[0] у нас слово, в i[1] - частота появления
  #reverse=1 говорить сортироваться по убыванию

  sortedWords = [] #Тут будет лист всех отсортированных слов

  #Проходим по всем словам в отсортированном списке
  for word in wordsList:
    sortedWords.append(word[0]) #Докидываем слово в лист отсортированных слов

  #Это словарь слово - индекс
  #Изначально заполнен всеми словами
  #У всех индекс 0
  wordIndexes = dict.fromkeys(allWords, 0)
  #Проходим по всем словам
  for word in wordIndexes.keys():
    wordIndexes[word] = sortedWords.index(word)+1 #Ставим индекс = индекс слова в отсортированном листе слов + 1
    #+1 потому, что индекс 0 резервируем под неопознанные слова

  return wordIndexes


###########################
# Преобразования листа слов в лист индексов
##########################
def words2Indexes(words, vocabulary, maxWordsCount):
  wordsIndexes = []
  
  #Идём по всем словая
  for word in words:
    
    wordIndex = 0 #Тут будет индекс слова, изначально 0 - слово неопознано
    wordInVocabulary = word in vocabulary #Проверяем, есть ли слово в словаре
    
    #Если слово есть в словаре
    if (wordInVocabulary):
      index = vocabulary[word] #Индекс = индексу слова в словаре
      if (index < maxWordsCount): #Если индекс ниже maxWordsCount - черты отсечения слов
        wordIndex = index #То записываем индекс
      #Иначе останется значение 0
        
    wordsIndexes.append(wordIndex)
    
  return wordsIndexes


###########################
# Преобразование одного короткого вектора в вектор из 0 и 1
# По принципу words bag
##########################
def changeXTo01(trainVector, wordsCount):
  #Создаём вектор длины wordsCount с нулями
  out = np.zeros(wordsCount)
  
  #Идём по всем индексам в строке
  for x in trainVector:
    out[x] = 1 #В позицию нужного индекса ставим 1
    
  return out


###########################
# Преобразование выборки (обучающей или проверочной) к виду 0 и 1
# По принципу words bag
##########################
def changeSetTo01(trainSet, wordsCount):
  out = []
  
  #Проходим по всем векторам в наборе
  for x in trainSet:
    out.append(changeXTo01(x, wordsCount)) #Добавляем в итоговый набор текущий вектор, преобразованный в bag of words
    
  return np.array(out)


###########################
# Формируем обучающую выборку из примечаний к квартирам
# Пока в виде слов
##########################
def getXTrainComments(data):
  xTrainComments = [] #Тут будет обучающся выборка
  allTextComments = "" #Тут будуте все тексты вместе для словаря
  
  #Идём по всем строкам квартир в базе
  for d in data:
    currText = getComment(d) #Вытаскиваем примечание к квартире
    try: 
      if (currText == currText): #Проверяем на nan
        allTextComments += currText + " " #Добавляем текст в общий текст для словаря
    except:
      currText = "Нет комментария" #Если не получается, то делаем стандартный текст "Нет комментария"
    xTrainComments.append(currText) #Добавляем примечание новой строкой в обучающую выборку
  
  xTrainComments = np.array(xTrainComments)
  
  return (xTrainComments, allTextComments)

###########################
# Формируем обучающую выборку из примечаний к квартирам
# Теперь в виде индексов
##########################
def changeSetToIndexes(xTrainComments, vocabulary, maxWordsCount):
  xTrainCommentsIndexes = [] #Тут будет итоговый xTrain примечаний в виде индексов
  
  #Идём по всем текстам
  for text in xTrainComments:
    currWords = text2Words(text) #Разбиваем текст на слова
    currIndexes = words2Indexes(currWords, vocabulary, maxWordsCount) #Превращаем в лист индексов
    currIndexes = np.array(currIndexes)
    xTrainCommentsIndexes.append(currIndexes) #Добавляем в xTrain
  
  xTrainCommentsIndexes = np.array(xTrainCommentsIndexes)
  xTrainCommentsIndexes = changeSetTo01(xTrainCommentsIndexes, maxWordsCount) #Превращаем в формат bag of words
  return xTrainCommentsIndexes

###########################
# Формируем обучающую выборку из примечаний к квартирам
# Теперь в виде индексов
# И с приведением к стандартной длине всех векторов - cropLen
##########################
def changeSetToIndexesCrop(xTrainComments, vocabulary, maxWordsCount, cropLen):
  xTrainCommentsIndexes = [] #Тут будет итоговый xTrain примечаний в виде индексов
  
  #Идём по всем текстам
  for text in xTrainComments:
    currWords = text2Words(text) #Разбиваем текст на слова
    currIndexes = words2Indexes(currWords, vocabulary, maxWordsCount) #Превращаем в лист индексов
    currIndexes = np.array(currIndexes)
    xTrainCommentsIndexes.append(currIndexes) #Добавляем в xTrain
  
  xTrainCommentsIndexes = np.array(xTrainCommentsIndexes)
  xTrainCommentsIndexes = pad_sequences(xTrainCommentsIndexes, maxlen=cropLen) #Приводим все вектора к стандартной длине
  return xTrainCommentsIndexes

##Собираем xTrain по текстам

*Разбор данного раздела:* https://youtu.be/PCMkFDuxvCI?t=6565


In [ ]:
xTrainC, allTextComments = getXTrainComments(data1) #Создаём обучающую выборку по текстам и большо текст для словаря
allWords = text2Words(allTextComments) #Собираем полный текст в слова
allWords = allWords[::10] #Берём 10% слов (иначе словарь слишком долго формируется)
vocabulary = createVocabulary(allWords) #Создаём словарь
xTrainC01 = changeSetToIndexes(xTrainC, vocabulary, 2000) #Преобразеум xTrain в bag of words

In [ ]:
#Выводим раземры обучающей выборки
#Чтобы проверить, что мы всё правильно собрали
print(xTrain.shape)
print(xTrainC01.shape)
print(yTrain.shape)

##Нормировка данных


In [ ]:
#Нормируем размер квартиры в xTrain
xScaler = StandardScaler() #Создаём нормировщик нормальным распределением
xScaler.fit(xTrain[:,-1].reshape(-1, 1)) #Обучаем его на площадях квартир (последня колонка в xTrain)
xTrainScaled = xTrain.copy()
xTrainScaled[:,-1] = xScaler.transform(xTrain[:,-1].reshape(-1, 1)).flatten() #Нормируем данные нормировщиком

#Выводим размер и два примера для сравнения
#Не нормированных данных и нормированных
print(xTrainScaled.shape)
print(xTrain[0])
print(xTrainScaled[0])

In [ ]:
#Нормируем выход сети - цену квартиры
yScaler = StandardScaler() #Делаемнормальный нормировщик
yScaler.fit(yTrain.reshape(-1, 1)) #Обучаем на ценах квартир
yTrainScaled = yScaler.transform(yTrain.reshape(-1, 1)) #Нормируем цены квартир

#Выводим размер и два примера для сравнения
#Не нормированных данных и нормированных
print(yTrainScaled.shape)
print(yTrain[0])
print(yTrainScaled[0])

In [ ]:
print(min(yTrain), max(yTrain))
print(min(yTrainScaled), max(yTrainScaled))

Формируем проверочную выборку

In [ ]:
splitVal = 0.2 #Процент, который выделяем в проверочную выборку
valMask = np.random.sample(xTrainScaled.shape[0]) < splitVal #Создаём маску True-False для создания проверочной выборки

In [ ]:
np.save('/content/drive/My Drive/Colab Notebooks/19 Регрессия/xTrainScaled',xTrainScaled)
np.save('/content/drive/My Drive/Colab Notebooks/19 Регрессия/xTrainC01',xTrainC01)
np.save('/content/drive/My Drive/Colab Notebooks/19 Регрессия/yTrainScaled',yTrainScaled)

In [ ]:
xTrainScaled = np.load('/content/drive/My Drive/Colab Notebooks/19 Регрессия/xTrainScaled.npy')
xTrainC01 = np.load('/content/drive/My Drive/Colab Notebooks/19 Регрессия/xTrainC01.npy')
yTrainScaled = np.load('/content/drive/My Drive/Colab Notebooks/19 Регрессия/yTrainScaled.npy')

##Нейронка

*Разбор данного раздела:* https://youtu.be/PCMkFDuxvCI?t=6728

In [ ]:
xTrainScaled.shape

In [ ]:
xTrainC01.shape

In [ ]:
trainableParams = [] #Лист тренируемых параметров

trainableParams.append(tf.Variable(tf.random.normal([109, 10], stddev=0.03), name='inp1')) 
trainableParams.append(tf.Variable(tf.random.normal([2000, 250], stddev=0.03), name='inp2')) 

trainableParams.append(tf.Variable(tf.random.normal([260, 100], stddev=0.03), name='first_hidden')) 
trainableParams.append(tf.Variable(tf.random.normal([100, 10], stddev=0.03), name='second_hidden')) 
trainableParams.append(tf.Variable(tf.random.normal([10, 1], stddev=0.03), name='out')) 

In [ ]:
def loss(pred , target):
    return tf.losses.mse(target , pred)

In [ ]:
#Полносвязный слой с несколькими функциями активации https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1291s
def base(x1, x2, params):
    inp1 = params[0]
    inp2 = params[1]

    first_hidden = params[2]
    second_hidden = params[3]
    out = params[4]

    lay_1 = tf.nn.relu(x1@inp1)
    lay_2 = tf.nn.relu(x2@inp2)

    lay_conc = tf.concat((lay_1, lay_2), axis=1)

    dens_1 = tf.nn.relu(lay_conc@first_hidden)
    dens_2 = tf.nn.relu(dens_1@second_hidden)
    y = dens_2@out

    return y

In [ ]:
#Модель  https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1541s
def model(x1, x2):
    y = base(x1, x2, trainableParams)
    return y 


In [ ]:
xTrainScaled[~valMask][[0]].shape

In [ ]:
#Посмотрим, какой выход у модели, если на вход подать случайную картинку 
model(xTrainScaled[~valMask][[0]], xTrainC01[~valMask][[0]]) #Смотрим вывод сети для первого изображения из MNIST https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1559s

In [ ]:
yTrainScaled[~valMask][0]

In [ ]:
#Функция для информативного вывода обучения https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1665s
#На вход получает:
# current - номер текущего батча
# amount - число всех батчей 
# params - словарь дополнительных параметров для вывода 

def print_log(current, amount, params):
  #Мы будем выводить прогрессбар, подобный прогрессбару при обучении нейронных сетей на Keras
  #Формат прогрессбара 23/120 [=====>------------------------]
  
  bar_len = 30 #Длина бара 
  percent = int(current * bar_len / amount) #Процент выполненной работы
  progressbar = ''

  for i in range(bar_len): #Проходим по всем элементам прогрессбара и добавляем символы в соответсвии с прогресом 
    if(i < percent):
      progressbar += '='
    elif(i == percent):
      progressbar += '>'
    else:
      progressbar += '-'

  #Добавялем в финальное сообщение символ переноса коретки консоли на начальную строку, добавляпем информацию о номере батча
  #количестве всех батчей, прогрессбар
  #Символ переноса коретки \r добавляется для того, чтобы каждый новый батч перезаписывать вывод. Таким образом вывод не будет засоряться повторяющейся информацией
  message = "\r" + str(current) + '/' + str(amount) + ' [' + progressbar + ']  ' 
  #Добавляем дополнительные параметры в вывод
  for key in params:
    message += key + str(params[key]) + '. '
  
  print(message, end='')

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #Задаем оптимизатор https://www.youtube.com/watch?v=ljMOTWrszxQ&t=1860s
m = tf.keras.metrics.MeanAbsoluteError() #Задаем метрику

def train(model, inputs, outputs): #Функция тренировки сети 

    with tf.GradientTape() as tape:
      current_loss = tf.reduce_mean(loss(model(inputs[0], inputs[1]), outputs)) #Считаем ошибку

      # Градиентный спуск. Инициализируем через learning rate
      # Функция реализует градиентный спуск и обратное распространение ошибки.
      grads = tape.gradient(current_loss , trainableParams)
      #Применение градиентного спуска
      optimizer.apply_gradients(zip(grads , trainableParams))
      #Подсчет точности сети 
      m.update_state(outputs, model(inputs[0], inputs[1]))
    return current_loss, m.result().numpy()

In [ ]:
!rm -R /tmp/mylogs/eager #Очищаем папку с записанной информацией обучения (если код запускается не в первый раз)

In [ ]:
batchSize = 128
epochs = 50

In [ ]:
path = '/tmp/mylogs/eager/' #Указываем путь для сохранения даных для Tensorboard https://www.youtube.com/watch?v=ljMOTWrszxQ&t=2237s


current_time = datetime.now().strftime("%Y%m%d-%H%M%S") #Получаем информацию о текущем времени для добавления данной информации к имени  
loss_log_dir = path + current_time + '/data' #Создаем папку с именем из текущего времени
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir) #Создаем средство записи файла резюме для данного каталога
x_len = len(xTrainScaled[~valMask])
amount_bathces = int(x_len / batchSize) #Считаем число батчей для каждой эпохи. Понадобится для вывода

with loss_summary_writer.as_default(): #Используя созданное средство для записи файла в резюме 
 
  for epoch in range(1, epochs + 1): #Проходим по каждой эпохе
    learningEpochStartTime = datetime.now() #Запоминаем время начала эпохи 
    print('Эпоха', epoch , '/', epochs) #Пишем текущую эпоху и общее число эпох
    avg_loss = 0 #Задаем среднюю ошибку

    for batch in range(0, x_len, batchSize): #Проходим по x_train с шагом batchSize
      current_loss, accuracy = train(model, [xTrainScaled[~valMask][batch: batch + batchSize],
                                             xTrainC01[~valMask][batch: batch + batchSize]],
                                     yTrainScaled[~valMask][batch: batch + batchSize]) #Тренируем сеть и получаем значение ошибки
      avg_loss += current_loss #Считаем среднюю ошибку

      #Задаем параметры, которые будем выводить 
      params = {'Время обучения на эпохе: ': datetime.now() - learningEpochStartTime, #Считаем время обучения на данной эпохе и добавляем в словарь
                'loss: ': round(current_loss.numpy(), 4), #Переводим ошибку в Numpy и добавляем в словарь
                'accuracy: ': round(accuracy, 4)} #Добавляем точность в словарь
      if(batch >= x_len - batchSize): #На последнем батче
        params['loss: '] = round((avg_loss / amount_bathces).numpy(), 4) #Выводим среднюю ошибку на всей эпохе

      current_batch = int(batch / batchSize) + 1 #Считаем номер текущего батча
      print_log(current_batch, amount_bathces, params) #Выводим всю нужную информацию 
    
    tf.summary.scalar("avg_loss", avg_loss, step=epoch) #Сохраняем данные для Tensorboard
    tf.summary.scalar("accuracy", accuracy, step=epoch) #Сохраняем данные для Tensorboard
    loss_summary_writer.flush() #Очищаем буфер вывода 
    print() #Вручную переносим каретку на следующую строку, чтобы не стирать финальные значения сети на эпохе

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir '/tmp/mylogs/eager/'

In [ ]:
# проверим на обучающей выборке
for _ in range(5):
  i = random.randint(0,len(xTrainScaled[~valMask]))
  outp = model(xTrainScaled[~valMask][[i]], xTrainC01[~valMask][[i]])
  outp = yScaler.inverse_transform(outp)[0][0]
  y_tr = yTrainScaled[~valMask][[i]]
  y_tr = yScaler.inverse_transform(y_tr)[0][0]
  print('предсказание= ', outp, 'факт= ', y_tr, abs(outp/y_tr*100-100),'%')

In [ ]:
# проверим на тестовой выборке
for _ in range(5):
  i = random.randint(0,len(xTrainScaled[valMask]))
  outp = model(xTrainScaled[valMask][[i]], xTrainC01[valMask][[i]])
  outp = yScaler.inverse_transform(outp)[0][0]
  y_tr = yTrainScaled[valMask][[i]]
  y_tr = yScaler.inverse_transform(y_tr)[0][0]
  print('предсказание= ', outp, 'факт= ', y_tr, abs(outp/y_tr*100-100),'%')